In [ ]:
%matplotlib inline


# Define a function with a field output: the viscous free fall example


In this example, we define a function which has a vector input and a field output. This is why we use the `PythonPointToFieldFunction` class to create the associated function and propagate the uncertainties through it.



## Introduction 

We consider an object inside a vertical cylinder which contains a viscous fluid. The fluid generates a drag force which limits the speed of the solid and we assume that the force depends linearily on the object speed:

\begin{align}m \frac{dv}{dt} = - m g - c v\end{align}


for any $t \in [0, t_{max}]$ where:

- $v$ is the speed $[m/s]$,
- $t$ is the time $[s]$,
- $t_{max}$ is the maximum time $[s]$,
- $g$ is the gravitational acceleration $[m.s^{-2}]$,
- $m$ is the mass $[kg]$,
- $c$ is the linear drag coefficient $[kg.s^{-1}]$.

The previous differential equation has the exact solution:

\begin{align}z(t) = z_0 + v_{inf} t + \tau (v_0 - v_{inf})\left(1 - e^{-\frac{t}{\tau}}\right)\end{align}


for any $t \in [0, t_{max}]$

where:

- $z$ is the altitude above the surface $[m]$,
- $z_0$ is the initial altitude $[m]$,
- $v_0$ is the initial speed (upward) $[m.s^{-1}]$,
- $v_{inf}$ is the limit speed $[m.s^{-1}]$:

\begin{align}v_{inf}=-\frac{m g}{c}\end{align}


- $\tau$ is time caracteristic $[s]$:

\begin{align}\tau=\frac{m}{c}.\end{align}


The stationnary speed limit at infinite time is equal to $v_{inf}$:

\begin{align}\lim_{t\rightarrow+\infty} v(t)= v_{inf}.\end{align}


When there is no drag, i.e. when $c=0$, the trajectory depends quadratically on $t$:

\begin{align}z(t) = z_0 + v_0 t -g t^2\end{align}


for any $t \in [0, t_{max}]$.

Furthermore when the solid touches the ground, we ensure that the altitude remains nonnegative i.e. the final altitude is:

\begin{align}y(t) = \max(z(t),0)\end{align}


for any $t \in [0, t_{max}]$.

## Probabilistic model

The parameters $z_0$, $v_0$, $m$ and $c$ are probabilistic:

- $z_0 \sim \mathcal{U}(100, 150)$,
- $v_0 \sim \mathcal{N}(55, 10)$,
- $m \sim \mathcal{N}(80, 8)$,
- $c \sim \mathcal{U}(0, 30)$.



## References

* Steven C. Chapra. Applied numerical methods with Matlab for engineers and scientists, Third edition. 2012. Chapter 7, "Optimization", p.182.



## Define the model



In [ ]:
from __future__ import print_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

We first define the time grid associated with the model. 



In [ ]:
tmin=0.0 # Minimum time
tmax=12. # Maximum time
gridsize=100 # Number of time steps
mesh = ot.IntervalMesher([gridsize-1]).build(ot.Interval(tmin, tmax))

The `getVertices` method returns the time values in this mesh.



In [ ]:
vertices = mesh.getVertices()
vertices[0:5]

Creation of the input distribution.



In [ ]:
distZ0 = ot.Uniform(100.0, 150.0)
distV0 = ot.Normal(55.0, 10.0)
distM = ot.Normal(80.0, 8.0)
distC = ot.Uniform(0.0, 30.0)
distribution = ot.ComposedDistribution([distZ0, distV0, distM, distC])

In [ ]:
dimension = distribution.getDimension()
dimension

Then we define the Python function which computes the altitude at each time value. In order to compute all altitudes with a vectorized evaluation, we first convert the vertices into a `numpy` `array` and use the `numpy` function `exp` and `maximum`: this increases the evaluation performance of the script. 



In [ ]:
def AltiFunc(X):
    g  = 9.81
    z0 = X[0]
    v0 = X[1]
    m  = X[2]
    c  = X[3]
    tau = m / c
    vinf = - m * g / c
    t = np.array(vertices)
    z = z0 + vinf * t + tau * (v0 - vinf) * (1 - np.exp( - t / tau))
    z = np.maximum(z,0.)
    return [[zeta[0]] for zeta in z]

In order to create a `Function` from this Python function, we use the `PythonPointToFieldFunction` class. Since the altitude is the only output field, the third argument `outputDimension` is equal to `1`. If we had computed the speed as an extra output field, we would have set `2` instead.



In [ ]:
outputDimension = 1
alti = ot.PythonPointToFieldFunction(dimension, mesh, outputDimension, AltiFunc)

## Sample trajectories



In order to sample trajectories, we use the `getSample` method of the input distribution and apply the field function.



In [ ]:
size = 10
inputSample = distribution.getSample(size)
outputSample = alti(inputSample)

In [ ]:
ot.ResourceMap.SetAsUnsignedInteger('Drawable-DefaultPalettePhase', size)

Draw some curves.



In [ ]:
graph = outputSample.drawMarginal(0)
graph.setTitle('Viscous free fall: %d trajectories' % (size))
graph.setXTitle(r'$t$')
graph.setYTitle(r'$z$')
view = viewer.View(graph)
plt.show()

We see that the object first moves up and then falls down. Not all objects, however, achieve the same maximum altitude. We see that some trajectories reach a higher maximum altitude than others. Moreover, at the final time $t_{max}$, one trajectory hits the ground: $z(t_{max})=0$ for this trajectory.

